Antes de comenzar con la ejecucion de las funciones, es necesario que las dependencias necesarias esten instaladas. Se recomienda crear un entorno virtual con los paquetes definidos en `requirements.txt` y ejecutar este Jupyter Notebook dentro de ese entorno.

In [1]:
# pip install memory-profiler==0.61.0

# Pregunta 1

1. Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días. Debe incluir las siguientes funciones:
```python
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
```
```python
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
```
```python
Returns: 
[(datetime.date(1999, 11, 15), "LATAM321"), (datetime.date(1999, 7, 15), "LATAM_CHI"), ...]
```

## Tiempo

In [2]:
from typing import List, Tuple
import datetime
import json
import os
import pandas as pd


def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    # Opening JSON file
    df = pd.read_json(path_or_buf=file_path, lines=True)
        

fp = "farmers-protest-tweets-2021-2-4.json"
q1_time(fp)

##### Usamos la funcion de  `Pandas` `read_json` que nos permite leer el archivo con formato JSON Line.

In [3]:
df = pd.read_json(path_or_buf=fp, lines=True)

##### Ya que los valores de la columna `user` son un objeto json, procedemos a extraer el campo `username`.

In [4]:
df['user'].iloc[0]

{'username': 'ArjunSinghPanam',
 'displayname': 'Arjun Singh Panam',
 'id': 45091142,
 'description': 'Global Citizen, Actor, Director: Sky is the roof above my head, the world is the road I travel, love is my food & mother earth is my bed. Roy in @CosmosMovie',
 'rawDescription': 'Global Citizen, Actor, Director: Sky is the roof above my head, the world is the road I travel, love is my food & mother earth is my bed. Roy in @CosmosMovie',
 'descriptionUrls': [],
 'verified': False,
 'created': '2009-06-06T07:50:57+00:00',
 'followersCount': 603,
 'friendsCount': 311,
 'statusesCount': 17534,
 'favouritesCount': 4269,
 'listedCount': 23,
 'mediaCount': 1211,
 'location': '',
 'protected': False,
 'linkUrl': 'https://www.cosmosmovieofficial.com',
 'linkTcourl': 'https://t.co/3uaoV3gCt3',
 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1215541746492461056/3De61YoQ_normal.jpg',
 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/45091142/1612601766',
 'url': 'https://twit

In [5]:
# Normalizar atributo 'username'
df['user'] = pd.json_normalize(df.user)['username']

##### Tambien necesitamos usar solo los valores de año, mes y dia de la columna `date`.

In [6]:
df['date'].iloc[0]

Timestamp('2021-02-24 09:23:35+0000', tz='UTC')

In [7]:
df['date'] = df['date'].dt.date
df['date'].iloc[0]

datetime.date(2021, 2, 24)

##### Agrupamos por fecha, contamos filas y se ordena en order descendente.

In [8]:
df2 = df[:]

group_by = ['date']
limit = 10


df2 = df2[['date', 'user']]
df2 = df2.groupby('date').count()
df2 = df2.sort_values('user', ascending=False)
df2 = df2.iloc[:limit, :]

df2 = df2.reset_index()

dates = df2['date'].to_list()
dates

[datetime.date(2021, 2, 12),
 datetime.date(2021, 2, 13),
 datetime.date(2021, 2, 17),
 datetime.date(2021, 2, 16),
 datetime.date(2021, 2, 14),
 datetime.date(2021, 2, 18),
 datetime.date(2021, 2, 15),
 datetime.date(2021, 2, 20),
 datetime.date(2021, 2, 23),
 datetime.date(2021, 2, 19)]

##### Ahora top usuario por dia


In [9]:
df2 = df[:]

top_users = []
for date in dates:
    df2 = df[:]
    where = df2['date'] == date
    group_by = ['user']
    limit = 10

    df2 = df2[where]
    df2 = df2[['date', 'user']]
    df2 = df2.groupby(group_by).count()
    df2 = df2.sort_values('date', ascending=False)
    df2 = df2.iloc[:limit, :]
    
    top_users.append(df2.index[0])
    
top_users

['RanbirS00614606',
 'MaanDee08215437',
 'RaaJVinderkaur',
 'jot__b',
 'rebelpacifist',
 'neetuanjle_nitu',
 'jot__b',
 'MangalJ23056160',
 'Surrypuria',
 'Preetm91']

##### De esta manera, se juntan ambas listas para formar el resultado deseado.

In [10]:
result = [(dates[i], top_users[i]) for i in range(10)]
    
result

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

#### Funcion corriendo con medicion de tiempo

In [11]:
from typing import List, Tuple
import datetime
import pandas as pd
import cProfile
import pstats

def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    # Leer archivo JSON
    df = pd.read_json(path_or_buf=file_path, lines=True)

    # Normalizar atributo 'username'
    df['user'] = pd.json_normalize(df.user)['username']

    # Mantener solo año, mes y dia - dateime.date(AAAA, MM, DD)
    df['date'] = df['date'].dt.date

    # - Paso 1. Obtener top 10 fechas con mas tweets

    # Crear nuevo DataFrame solo con las columnas 'date' y 'user'
    df_dates = df[['date', 'user']]

    # Agrupar por fecha y contar filas
    df_dates = df_dates.groupby('date').count()

    # Ordenar por conteo de filas de manera descendente
    df_dates = df_dates.sort_values('user', ascending=False)

    # Elegir solo los primeros 10 resultados
    df_dates = df_dates.iloc[:10, :]

    top_dates = df_dates.index.to_list()

    # - Paso 2. Obtener top usuario con mas tweets por dia

    top_users = []
    # Iterar por cada dia
    for date in top_dates:
        # Crear nuevo DataFrame son con las columnas `data` y `user`
        df_users = df[['date', 'user']]

        # Seleccionar filas solo con la fecha a considerar
        mask = df_users['date'] == date
        df_users = df_users[mask]

        # Agrupar por usuario y contar filas
        df_users = df_users.groupby('user').count()

        # Ordenar por conteo de filas de manera descendente
        df_users = df_users.sort_values('date', ascending=False)

        # Guardar usuario con mas tweets
        top_users.append(df_users.index[0])

    # Juntar en una lista de 'Tuples' los top 10 dias
    # con su respectivo usuario con mas tweets
    result = [(top_dates[i], top_users[i]) for i in range(10)]

    return result


with cProfile.Profile() as profile:
    basePath = os.path.abspath('')
    fp = os.path.join(basePath, "farmers-protest-tweets-2021-2-4.json")
    q1_time(fp)

results = pstats.Stats(profile)
results.sort_stats(pstats.SortKey.TIME)
results.print_stats()

         6886310 function calls (6767961 primitive calls) in 10.776 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    4.337    4.337    4.355    4.355 {built-in method pandas._libs.json.ujson_loads}
   352223    1.163    0.000    1.166    0.000 {method 'items' of 'dict' objects}
   117407    0.464    0.000    0.649    0.000 C:\Users\USER\Documents\Git Projects\latam-challenge\venv\lib\site-packages\pandas\io\json\_normalize.py:182(<dictcomp>)
        2    0.449    0.224    0.692    0.346 C:\Users\USER\Documents\Git Projects\latam-challenge\venv\lib\site-packages\pandas\core\internals\construction.py:891(_list_of_dict_to_arrays)
5179220/5179062    0.384    0.000    0.385    0.000 {built-in method builtins.isinstance}
        1    0.377    0.377    0.502    0.502 {method 'read' of '_io.TextIOWrapper' objects}
       42    0.320    0.008    0.322    0.008 C:\Users\USER\Documents\Git Projects\latam-challenge\venv\lib

## Memoria

##### Ahora, para lograr el mismo resultado siendo eficientes con la memoria, ya no se cargara todo el archivo al mismo tiempo. Si no que se leera y procesara por partes.

In [1]:
from typing import List, Tuple
import datetime
import pandas as pd
import os
from memory_profiler import profile, memory_usage


def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    
    # El archivo JSON Lines se lee y procesa en chunks de tamaño 'chunksize' a la vez
    lines = pd.read_json(file_path, lines=True, chunksize=1000)

    dates = {}
    users = {}

    for df in lines:
        # El stream envia los index originales del DataFrame y esto
        # estorba el procesamiento aislado de cada DataFrame

        df.reset_index(inplace=True)
        # 1. Contar fechas

        # Mantener solo año, mes y dia - dateime.date(AAAA, MM, DD)
        df['date'] = df.loc[:, 'date'].dt.date

        for date, val in df['date'].value_counts().items():
            dates[date] = dates.get(date, 0) + val

        # 2. Contar usuarios
        # Elegir solo las filas de interes - date y user,
        # y una tercera para contar
        df_users = df[['date', 'user', 'id']]

        # Normalizar usuario
        df_users.loc[:, 'user'] = pd.json_normalize(df_users['user'])['username']

        # Agrupar fecha y usuario
        df_users = df_users.groupby(['date', 'user']).count()

        # Ordenar por fecha y numero de tweets por usuario
        df_users = df_users.sort_values(['date', 'id'], ascending=False)

        # Crear un diccionario donde se va sumando el conteo de cada usuario por fecha
        # y asi sacar al usuario con mas tweets dentro las top 10 fechas
        for index, value in df_users.iterrows():
            date = index[0]
            user = index[1]
            val = value.iloc[0]

            if date in users.keys():
                if user in users[date].keys():
                    # Usuario y fecha existen en el diccionario, asi que se suma
                    users[date][user] += val

                else:
                    # Crear entrada de usuario
                    users[date][user] = val
            else:
                # Crear entrada de fecha
                users[date] = {user: val}

    # Crear DataFrame para ordenar las fechas en orden descendente
    dates = pd.DataFrame.from_dict(
        {'date': dates.keys(), 'count': dates.values()})
    dates = dates.sort_values(['count'], ascending=False).iloc[:10, :]

    # Lista con las top 10 fechas con mas tweets
    top_dates = list(dates['date'])

    # Elegir los usuarios con mas tweets dentro de las fechas de interes
    top_users = [max(users[date], key=users[date].get) for date in top_dates]

    # Juntar en una lista de 'Tuples' los top 10 dias
    # con su respectivo usuario con mas tweets

    result = [(top_dates[i], top_users[i]) for i in range(10)]

    return result


if __name__ == '__main__':
    basePath = os.path.abspath('')
    fp = os.path.join(basePath, "farmers-protest-tweets-2021-2-4.json")

    mem = memory_usage((q1_memory, (), {'file_path': fp}), max_usage=True)

    print('Maximo uso de memoria: ', mem, ' MiB')

Maximo uso de memoria:  137.30859375  MiB
